<h1>
<center>Graph Construction </center>
</h1>

### Generals 

Dependencies

In [1]:
#Data
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import shutil

#Graph Counstruction
import torch
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt


import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN2
from torch.nn import Linear
from torch.nn import ReLU
import torch.nn as nn
from torch.nn.init import kaiming_uniform_

/opt/homebrew/Caskroom/miniforge/base/envs/full_ml/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Envariables

In [2]:
train_path = 'io/input/data_raw/train.csv'
test_path = 'io/input/data_raw/example_test.csv'
assets_details_path ='io/input/data_raw/asset_details.csv'
data_intermediate_path ='io/input/data_intermediate/'
train_obj_path = data_intermediate_path + 'train_object.pkl'
train_node_features_path = data_intermediate_path + 'node_features/train/'
train_node_labels_path = data_intermediate_path + 'node_labels/train/'
test_node_labels_path = data_intermediate_path + 'node_labels/test/'
test_node_features_path = data_intermediate_path + 'node_features/test/'
plot_export_path = 'io/output/exports/analysis_plots/'
chunk_size = 100000
timesteps = 1200

### Core Functionality

In [132]:
def save_object(arr,chunk_size,path):
    num_chunks = arr.shape[0] // chunk_size + 1
    shutil.rmtree(path)
    os.makedirs(path)
    for i in tqdm(range(num_chunks)):
        chunk = arr[i*chunk_size:(i+1)*chunk_size, :]
        filename = f"{path}chunk_{i}.npy"
        np.save(filename, chunk)
    return True

In [133]:
def load_object(path):
    num_chunks = len([f for f in os.listdir(path) if f.startswith('chunk_') and f.endswith('.npy')])
    # Load array from chunks
    chunks = []
    for i in range(num_chunks):
        filename = f"{path}chunk_{i}.npy"
        print()
        chunk = np.load(filename,allow_pickle=True)
        chunks.append(chunk)
    arr_reconstructed = np.concatenate(chunks, axis=0)
    return arr_reconstructed

In [134]:
def load_data(train_path,test_path,assets_details_path):
    trainset = pd.read_csv(train_path,index_col=0)
    testset = pd.read_csv(test_path,index_col=0)
    assets_details = pd.read_csv(assets_details_path,index_col=0)
    return trainset,testset,assets_details

In [135]:
def intial_preprocessing(df,assets_details,mode):
    df.reset_index(inplace=True)
    assets_details.reset_index(inplace=True)
    
    df = pd.merge(df, assets_details[['Asset_ID', 'Weight', 'Asset_Name']],
                         on='Asset_ID', how='inner', left_index=False)
    
    df = df.sort_values('timestamp', ascending=False)
    df = reduce_mem_usage(df)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.drop(['Asset_ID'], axis=1)
    if mode == 'train':
        df['Target'].fillna(df['Target'].median(), inplace=True) #Must Change Approach
    elif mode == 'test':
        df = df.drop(['group_num'], axis=1)
        df = df.drop(['row_id'], axis=1)
    df = df[df['timestamp'].dt.minute == 0]
    return df

In [136]:
def intial_preprocessing_pivot(df,assets_details):
    df.reset_index(inplace=True)
    assets_details.reset_index(inplace=True)
    df = pd.merge(df, assets_details[['Asset_ID', 'Weight', 'Asset_Name']],
                         on='Asset_ID', how='inner', left_index=False)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.drop(['Asset_ID'], axis=1)
    df = df[df['timestamp'].dt.time == pd.to_datetime('08:00').time()]
    pivot = df.pivot_table(index='timestamp', columns='Asset_Name', values='Close')
    pivot = pivot.dropna()
    return pivot

In [137]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    return df

### Shape Convertion

In [138]:
def node_features_construction(df,timesteps):
    feature_names = df.columns
    # Convert the 'timestamp' column to a pandas datetime object
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Get a list of all unique asset IDs in the dataset
    asset_ids = df['Asset_Name'].unique()
    timestamps = df['timestamp'].unique()

    # Get a list of all features in the dataset
    features = list(df.columns)
    features.remove('timestamp')
    features.remove('Asset_Name')
    results=[]
    results=[0 for i in range(len(timestamps))]

    for i in tqdm( range (0,len(timestamps))):
        results[i] = df[df["timestamp"]==timestamps[i]]
        results[i] = results[i].drop(['timestamp'], axis=1)
        results[i] = results[i].drop(['Asset_Name'], axis=1)
    
    
    results = results[:timesteps]
    np_result = np.array(results)
    print(np_result.shape)
    np_result = np.reshape(np_result, (len(results),len(asset_ids),len(features),1))
    
    print('Array Tranformation Finished!')
    print(np_result.shape)
    return np_result

In [139]:
def node_labels_construction(df,timsteps):
    asset_ids = df['Asset_Name'].unique()
    timestamps = df['timestamp'].unique()
    timestamps = timestamps[:timsteps]
    df = df.loc[df['timestamp'].isin(timestamps)].reset_index(drop=True)
    result = df.pivot(index='timestamp', columns='Asset_Name', values='Target')
    result = np.array(result)
    result = np.reshape(result, (len(result), len(asset_ids),1))
    return result

In [140]:
def adjency_matrixes_granger_test(df_pivot):
    df_pivot = df_pivot.tail(100)
    # Create an empty list to store the adjacency matrices
    adj_matrices = []

    # Loop through each timestamp
    for i in tqdm (range (len(df_pivot))):

        # Select the data up to the current timestamp
        data = df_pivot.iloc[:i+1,:]
        
        # Create an empty adjacency matrix for the current timestamp
        adj_matrix = np.zeros((len(df_pivot.columns), len(df_pivot.columns)))

        # Loop through each combination of asset IDs
        for j in range(len(df_pivot.columns)):
            for k in range(len(df_pivot.columns)):

                # Skip if j and k are the same asset ID
                if j == k:
                    continue

                # Select the data for the current pair of asset IDs
                data_jk = data.iloc[:,[j,k]].dropna()

                # Skip if there is no data for the current pair of asset IDs
                if len(data_jk) == 0:
                    continue

                # Calculate Granger causality in both directions between j and k
                try:
                    result1 = grangercausalitytests(data_jk, maxlag=10, verbose=False)
                    max_F_statistic1 = max(result1[lag][0]['params_ftest'][0] for lag in result1.keys())
                    result2 = grangercausalitytests(data_jk.iloc[:,::-1], maxlag=10, verbose=False)
                    max_F_statistic2 = max(result2[lag][0]['params_ftest'][0] for lag in result2.keys())
                except:
                    max_F_statistic1 = None
                    max_F_statistic2 = None

                # Add the Granger causality values to the adjacency matrix
                if max_F_statistic1 is not None and max_F_statistic2 is not None:
                    adj_matrix[j,k] = max_F_statistic1
                    adj_matrix[k,j] = max_F_statistic2

        # Add the current adjacency matrix to the list of adjacency matrices
        adj_matrices.append(adj_matrix)
    return adj_matrices,df_pivot.columns

In [225]:
def normalize_adj(adjency_matrixes):
    norm_adjency_matrixes = []
    for adj in tqdm(adjency_matrixes):
        # Create a copy of the input matrix
        adj_norm = np.copy(adj)
        # Get the diagonal indices
        diag_indices = np.diag_indices(adj.shape[0])
        # Set the diagonal elements to 1
        adj_norm[diag_indices] = 1
        # Normalize the non-diagonal elements on the range 0.1-0.9
        non_diag_mask = np.ones(adj.shape, dtype=bool)
        non_diag_mask[diag_indices] = False
        adj_norm[non_diag_mask] = 0.1 + 0.8 * (adj_norm[non_diag_mask] - np.min(adj_norm[non_diag_mask])) / (np.max(adj_norm[non_diag_mask]) - np.min(adj_norm[non_diag_mask]))
        norm_adjency_matrixes.append(adj_norm)
    norm_adjency_matrixes = np.array(norm_adjency_matrixes)
    return norm_adjency_matrixes

In [192]:
def plot_adjency_matrixes(adj_matrix1,adj_matrix2,labels,timestamp1,timestamp2,plot_export_path):
    # Create two random adjacency matrices
    labels = asset_ids
    # Set up the figure and two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    # Plot the first adjacency matrix
    im1 = ax1.imshow(adj_matrix1, cmap='coolwarm', interpolation='None')
    ax1.set_xticks(np.arange(len(labels)))
    ax1.set_yticks(np.arange(len(labels)))
    ax1.set_xticklabels(labels,rotation=90)
    ax1.set_yticklabels(labels)
    ax1.tick_params(axis='both', labelsize=12)
    ax1.set_title(timestamp1)
    # Plot the second adjacency matrix
    im2 = ax2.imshow(adj_matrix2, cmap='coolwarm', interpolation='None')
    ax2.set_xticks(np.arange(len(labels)))
    ax2.set_yticks(np.arange(len(labels)))
    ax2.set_xticklabels(labels,rotation=90)
    ax2.set_yticklabels(labels)
    ax2.tick_params(axis='both', labelsize=12)
    ax2.set_title(timestamp2)
    # Add the matrix values as text to the plots
    for ax, adj_matrix in zip([ax1, ax2], [adj_matrix1, adj_matrix2]):
        for i in range(adj_matrix.shape[0]):
            for j in range(adj_matrix.shape[1]):
                text = ax.text(j, i, round(adj_matrix[i, j],1),
                               ha="center", va="center", color="w", fontsize=12)
    # Set the title of the figure
    fig.suptitle("Dynamic Graph Edge Info", fontsize=18)
    # Adjust the spacing between subplots
    fig.tight_layout(pad=2)
    # Display the plot
    plt.savefig(plot_export_path + 'Adjency_Matrixes.pdf')
    plt.show()

### Create and saved graph info

In [143]:
trainset,testset,assets_details = load_data(train_path,test_path,assets_details_path)

print('intial_preprocessing')
trainset = intial_preprocessing(trainset,assets_details,mode='train')
testset = intial_preprocessing(testset,assets_details,mode='test')

print('train_node_features')
train_node_features = node_features_construction(trainset,timesteps)
save_object(train_node_features,chunk_size,train_node_features_path)
del(train_node_features)

print('train_node_labels')
train_node_labels = node_labels_construction(trainset,timesteps)
save_object(train_node_labels,chunk_size,train_node_labels_path)
del(train_node_labels)
del(trainset)

print('test_node_features')
test_node_features = node_features_construction(testset,timesteps)
save_object(test_node_features,chunk_size,test_node_features_path)
del(test_node_features)

In [253]:
def adj_to_edge_lists(norm_adjency_matrixes):
    undirected_edges_list = []
    undirected_weights_list = [] 
    
    for adj_matrix in tqdm(norm_adjency_matrixes):
        edges = np.transpose(np.where(adj_matrix != 0))
        edge_weights = adj_matrix[edges[:, 0], edges[:, 1]]
        edges = [list(edges[:, 0] + 1), list(edges[:, 1] + 1)]
        edg_weights = list(edge_weights)
        undirected_edges_list.append(edges)
        undirected_weights_list.append(edge_weights)
    
    undirected_edges_list = np.array(undirected_edges_list)
    undirected_weights_list = np.array(undirected_weights_list)
    return undirected_edges_list,undirected_weights_list

### Edges 

In [257]:
timestamp1 = "2019/01/05  12:00"
timestamp2 = "2021/01/05  12:00"
trainset,testset,assets_details = load_data(train_path,test_path,assets_details_path)
pivot_ts = intial_preprocessing_pivot(trainset,assets_details)
adjency_matrixes,asset_ids = adjency_matrixes_granger_test(pivot_ts)
norm_adjency_matrixes = normalize_adj(adjency_matrixes)
#plot_adjency_matrixes(norm_adjency_matrixes[50],norm_adjency_matrixes[99],asset_ids,timestamp1,timestamp2,plot_export_path)
undirected_edges, undirected_weights = adj_to_edge_lists(norm_adjency_matrixes)

  0%|                                                   | 0/100 [00:00<?, ?it/s]/var/folders/7v/c_fky0td2dldmltq3618f3jh0000gn/T/ipykernel_14050/2537035348.py:13: RuntimeWarning: invalid value encountered in divide
  adj_norm[non_diag_mask] = 0.1 + 0.8 * (adj_norm[non_diag_mask] - np.min(adj_norm[non_diag_mask])) / (np.max(adj_norm[non_diag_mask]) - np.min(adj_norm[non_diag_mask]))
100%|██████████████████████████████████████| 100/100 [00:00<00:00, 26454.14it/s]
